In [1]:
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input,Dense,LSTM,Embedding,Dropout,add

In [10]:
gpus = tf.config.list_physical_devices("GPU")
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]


In [3]:
BASE_DIR='./'
WORKING_DIR='./'

Extract Image Features

In [5]:
model=VGG16()


In [6]:
model=Model(inputs=model.inputs,outputs=model.layers[-2].output)

In [7]:
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [9]:
features={}
directory=os.path.join(BASE_DIR,'Images')
for img_name in tqdm(os.listdir(directory)):
    img_path=directory+ '/'+img_name
    image=load_img(img_path,target_size=(224,224))
    # convert to array
    image=img_to_array(image)
    # reshape data
    image=image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
    # preprocess for VGG model
    image=preprocess_input(image)
    # extract features
    feature=model.predict(image,verbose=0)
    # img ID
    img_id= img_name.split('.')[0]
    features[img_id]=feature

  0%|          | 0/8091 [00:00<?, ?it/s]

In [11]:
pickle.dump(features,open(os.path.join(WORKING_DIR,'features.pkl'),'wb'))

In [12]:
with open(os.path.join(WORKING_DIR,'features.pkl'),'rb') as f:
    features=pickle.load(f)

In [ ]:
with open(os.path.join(BASE_DIR,'captions.txt'),'r') as f:
    # to skip first line
    next(f)
    captions_doc=f.read()